In [1]:
import pandas as pd
import numpy as np
import random

# Generate some random data for demonstration purposes
np.random.seed(42)
random.seed(42)

# Create a date range
dates = pd.date_range(start='2023-01-01', end='2023-10-01', freq='B')

# Create a dataframe with random stock data
df = pd.DataFrame(index=dates)

# Generate random stock prices for three hypothetical stocks
df['Stock_A'] = np.random.randint(80, 120, size=len(dates)).astype(float)
df['Stock_B'] = np.random.randint(50, 100, size=len(dates)).astype(float)
df['Stock_C'] = np.random.randint(100, 150, size=len(dates)).astype(float)

# Add some random noise to simulate price fluctuations
df += np.random.normal(0, 2, size=(len(dates), 3))

# Calculate daily returns
df['Returns_A'] = df['Stock_A'].pct_change()
df['Returns_B'] = df['Stock_B'].pct_change()
df['Returns_C'] = df['Stock_C'].pct_change()

# Calculate 5-day and 20-day rolling averages
df['Rolling_Avg_5'] = df['Returns_A'].rolling(window=5).mean()
df['Rolling_Avg_20'] = df['Returns_A'].rolling(window=20).mean()

df['Rolling_Avg_5_B'] = df['Returns_B'].rolling(window=5).mean()
df['Rolling_Avg_20_B'] = df['Returns_B'].rolling(window=20).mean()

df['Rolling_Avg_5_C'] = df['Returns_C'].rolling(window=5).mean()
df['Rolling_Avg_20_C'] = df['Returns_C'].rolling(window=20).mean()

# Create some random categorical signals for demonstration
df['Signal_A'] = np.where(df['Rolling_Avg_5'] > df['Rolling_Avg_20'], 1, 0)
df['Signal_B'] = np.where(df['Rolling_Avg_5_B'] > df['Rolling_Avg_20_B'], 1, 0)
df['Signal_C'] = np.where(df['Rolling_Avg_5_C'] > df['Rolling_Avg_20_C'], 1, 0)

# Display the dataframe
print(df.head())


               Stock_A    Stock_B     Stock_C  Returns_A  Returns_B  \
2023-01-02  117.975506  69.205491  114.151609        NaN        NaN   
2023-01-03  106.645677  61.950239  141.705885  -0.096035  -0.104836   
2023-01-04   92.349006  96.357228  136.825863  -0.134058   0.555397   
2023-01-05   85.872551  63.355559  132.487374  -0.070130  -0.342493   
2023-01-06  100.489933  80.986114  106.057923   0.170222   0.278280   

            Returns_C  Rolling_Avg_5  Rolling_Avg_20  Rolling_Avg_5_B  \
2023-01-02        NaN            NaN             NaN              NaN   
2023-01-03   0.241383            NaN             NaN              NaN   
2023-01-04  -0.034438            NaN             NaN              NaN   
2023-01-05  -0.031708            NaN             NaN              NaN   
2023-01-06  -0.199487            NaN             NaN              NaN   

            Rolling_Avg_20_B  Rolling_Avg_5_C  Rolling_Avg_20_C  Signal_A  \
2023-01-02               NaN              NaN            

In [2]:
# Sample Python Code for LSTM Model Training using Yahoo Finance data
!pip install yfinance  # Install yfinance package

import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Download stock data from Yahoo Finance
ticker = "AAPL"  # Example: Apple Inc.
start_date = "2010-01-01"
end_date = "2020-01-01"
data = yf.download(ticker, start=start_date, end=end_date)

# Check if 'Close' column is available
if 'Close' not in data.columns:
    raise ValueError("Data does not contain 'Close'. Please check the ticker and date range.")

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1))

# Prepare training data
training_data = scaled_data[:int(0.8*len(scaled_data))]
x_train, y_train = [], []
for i in range(60, len(training_data)):
    x_train.append(training_data[i-60:i, 0])
    y_train.append(training_data[i, 0])
x_train, y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

# Build and train the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(x_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(x_train, y_train, epochs=10, batch_size=32)

# Optionally, save the model
model.save('lstm_stock_model.h5')


[*********************100%%**********************]  1 of 1 completed


Epoch 1/10
61/61 [==============================] - 7s 42ms/step - loss: 0.0071
Epoch 2/10
61/61 [==============================] - 2s 41ms/step - loss: 1.8797e-04
Epoch 3/10
61/61 [==============================] - 2s 41ms/step - loss: 1.7563e-04
Epoch 4/10
61/61 [==============================] - 4s 68ms/step - loss: 1.6900e-04
Epoch 5/10
61/61 [==============================] - 3s 46ms/step - loss: 1.6033e-04
Epoch 6/10
61/61 [==============================] - 3s 41ms/step - loss: 1.5388e-04
Epoch 7/10
61/61 [==============================] - 3s 41ms/step - loss: 1.5806e-04
Epoch 8/10
61/61 [==============================] - 3s 46ms/step - loss: 1.5110e-04
Epoch 9/10
61/61 [==============================] - 4s 62ms/step - loss: 1.5854e-04
Epoch 10/10
61/61 [==============================] - 2s 41ms/step - loss: 1.3789e-04


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import yfinance as yf
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Fetch historical stock data using yfinance
ticker = 'AAPL'  # Example ticker symbol for Apple Inc.
data = yf.download(ticker, start='2020-01-01', end='2023-01-01')

# Calculate Simple Moving Average (SMA), Relative Strength Index (RSI), and Moving Average Convergence Divergence (MACD)
data['SMA'] = data['Close'].rolling(window=14).mean()
data['RSI'] = data['Close'].diff().apply(lambda x: np.where(x > 0, x, 0)).rolling(window=14).mean() / \
              data['Close'].diff().abs().rolling(window=14).mean() * 100
exp1 = data['Close'].ewm(span=12, adjust=False).mean()
exp2 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = exp1 - exp2

# Volume as a feature
data['Volume'] = data['Volume']

# Create binary labels for trading signals (1 for Buy, -1 for Sell)
data['Signal'] = np.where(data['Close'] > data['Close'].shift(1), 1, -1)

# Prepare data for training, dropping NaN values created by moving averages
data.dropna(inplace=True)
features = data[['SMA', 'RSI', 'MACD', 'Volume']]
X_train, X_test, y_train, y_test = train_test_split(features, data['Signal'], test_size=0.2, random_state=42)

# Build and train the GBM model
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
predictions = model.predict(X_test)

# Evaluate model accuracy
accuracy = accuracy_score(y_test, predictions)
print(f"Model Accuracy: {accuracy}")


[*********************100%%**********************]  1 of 1 completed


Model Accuracy: 0.5167785234899329


In [2]:
import pandas as pd
import yfinance as yf

# Collect historical stock price data for Apple Inc. (AAPL)
stock_data = yf.download('AAPL', start='2022-01-01', end='2023-01-01')

# Display the first few rows of the data
print(stock_data.head())

# Handle missing values by dropping them
stock_data = stock_data.dropna()

# Normalize the data using Min-Max scaling
stock_data_normalized = (stock_data - stock_data.min()) / (stock_data.max() - stock_data.min())

# Display the first few rows of the normalized data
print(stock_data_normalized.head())

[*********************100%%**********************]  1 of 1 completed

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
2022-01-03  177.830002  182.880005  177.710007  182.009995  179.481125   
2022-01-04  182.630005  182.940002  179.119995  179.699997  177.203201   
2022-01-05  179.610001  180.169998  174.639999  174.919998  172.489639   
2022-01-06  172.699997  175.300003  171.639999  172.000000  169.610199   
2022-01-07  172.889999  174.139999  171.029999  172.169998  169.777878   

               Volume  
Date                   
2022-01-03  104487900  
2022-01-04   99310400  
2022-01-05   94537600  
2022-01-06   96904000  
2022-01-07   86709100  
                Open      High       Low     Close  Adj Close    Volume
Date                                                                   
2022-01-03  0.912152  0.998868  0.973521  1.000000   1.000000  0.470076
2022-01-04  1.000000  1.000000  1.000000  0.958728   0.958181  0.434951
2022-01-05  0.944729  0.9